# Initialization

* implement unifyGNN+Contrastive
* try different noisy and augmentation
* try against with/without unnoticeable 

In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[1]: 

import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI


# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--model', type=str, default='GCN', help='model',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--dataset', type=str, default='Pubmed', 
                    help='Dataset',
                    choices=['Cora','Citeseer','Pubmed','PPI','Flickr','ogbn-arxiv','Reddit','Reddit2','Yelp'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=128,
                    help='Number of hidden units of backdoor model.')
parser.add_argument('--num_hidden', type=int, default=128,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=128,
                    help='Number of hidden units in MLP.')
parser.add_argument('--thrd', type=float, default=0.5)
parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--epochs', type=int,  default=200, help='Number of epochs to train benign and backdoor model.')
parser.add_argument('--trojan_epochs', type=int,  default=400, help='Number of epochs to train trigger generator.')
parser.add_argument('--inner', type=int,  default=1, help='Number of inner')
parser.add_argument('--temperature', type=float,  default=0.5, help='Temperature')
# backdoor setting
parser.add_argument('--lr', type=float, default=0.001,
                    help='Initial learning rate.')
parser.add_argument('--trigger_size', type=int, default=3,
                    help='tirgger_size')
parser.add_argument('--use_vs_number', action='store_true', default=True,
                    help="if use detailed number to decide Vs")
parser.add_argument('--vs_ratio', type=float, default=0,
                    help="ratio of poisoning nodes relative to the full graph")
parser.add_argument('--vs_number', type=int, default=0,
                    help="number of poisoning nodes relative to the full graph")
# defense setting
parser.add_argument('--defense_mode', type=str, default="none",
                    choices=['prune', 'isolate', 'none'],
                    help="Mode of defense")
parser.add_argument('--prune_thr', type=float, default=0.2,
                    help="Threshold of prunning edges")
parser.add_argument('--target_loss_weight', type=float, default=1,
                    help="Weight of optimize outter trigger generator")
parser.add_argument('--homo_loss_weight', type=float, default=0,
                    help="Weight of optimize similarity loss")
parser.add_argument('--homo_boost_thrd', type=float, default=0.5,
                    help="Threshold of increase similarity")
# attack setting
parser.add_argument('--dis_weight', type=float, default=1,
                    help="Weight of cluster distance")
parser.add_argument('--selection_method', type=str, default='cluster_degree',
                    choices=['loss','conf','cluster','none','cluster_degree'],
                    help='Method to select idx_attach for training trojan model (none means randomly select)')
parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
parser.add_argument('--evaluate_mode', type=str, default='1by1',
                    choices=['overall','1by1'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=2,
                    help="Threshold of prunning edges")
# GRACE setting
# parser.add_argument('--gpu_id', type=int, default=0)
parser.add_argument('--config', type=str, default="config.yaml")
## Contrasitve setting
parser.add_argument('--cl_lr', type=float, default=0.0005)
# parser.add_argument('--cl_num_hidden', type=int, default=128)
parser.add_argument('--cl_num_proj_hidden', type=int, default=128)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=1)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0.2)
parser.add_argument('--drop_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_feat_rate_1', type=float, default=0.3)
parser.add_argument('--drop_feat_rate_2', type=float, default=0.2)
parser.add_argument('--tau', type=float, default=0.1)
parser.add_argument('--cl_num_epochs', type=int, default=200)
parser.add_argument('--cl_weight_decay', type=float, default=1e-5)
parser.add_argument('--cont_batch_size', type=int, default=0)
parser.add_argument('--noisy_level', type=float, default=0.3)
parser.add_argument('--clf_weight', type=float, default=1)
parser.add_argument('--inv_weight', type=float, default=1)
# parser.add_argument('--select_thrh', type=float, default=0.8)

# Attack
parser.add_argument('--attack', type=str, default='nettack')
# Randomized Smoothing
parser.add_argument('--prob', default=0.8, type=float,
                    help="probability to keep the status for each binary entry")

# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)
#%%
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

if(args.dataset == 'Cora' or args.dataset == 'Citeseer' or args.dataset == 'Pubmed'):
    dataset = Planetoid(root='./data/', \
                        name=args.dataset,\
                        transform=transform)
elif(args.dataset == 'Flickr'):
    dataset = Flickr(root='./data/Flickr/', \
                    transform=transform)
elif(args.dataset == 'Reddit2'):
    dataset = Reddit2(root='./data/Reddit2/', \
                    transform=transform)
elif(args.dataset == 'ogbn-arxiv'):
    from ogb.nodeproppred import PygNodePropPredDataset
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = PygNodePropPredDataset(name = 'ogbn-arxiv', root='./data/')
    split_idx = dataset.get_idx_split() 

data = dataset[0].to(device)

# from deeprobust.graph.data import Dataset, Dpr2Pyg, Pyg2Dpr
# dr_data = Dataset(root='/tmp/', name='pubmed') # load clean graph
# data = Dpr2Pyg(dr_data)
# data = data[0].to(device)

if(args.dataset == 'ogbn-arxiv'):
    nNode = data.x.shape[0]
    setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
    # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
    data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.y = data.y.squeeze(1)
# we build our own train test split 
#%% 
# from utils import get_split
# # data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)
idx_train = data.train_mask.nonzero().flatten()
idx_val = data.val_mask.nonzero().flatten()
idx_clean_test = data.test_mask.nonzero().flatten()

from torch_geometric.utils import to_undirected
from utils import subgraph
data.edge_index = to_undirected(data.edge_index)
train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()

Namespace(add_edge_rate_1=0, add_edge_rate_2=0, attack='nettack', cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.0005, cl_num_epochs=200, cl_num_layers=2, cl_num_proj_hidden=128, cl_weight_decay=1e-05, clf_weight=1, config='config.yaml', cont_batch_size=0, cont_weight=1, cuda=True, dataset='Pubmed', debug=True, defense_mode='none', device_id=2, dis_weight=1, drop_edge_rate_1=0.2, drop_edge_rate_2=0, drop_feat_rate_1=0.3, drop_feat_rate_2=0.2, dropout=0.5, epochs=200, evaluate_mode='1by1', hidden=128, homo_boost_thrd=0.5, homo_loss_weight=0, inner=1, inv_weight=1, lr=0.001, model='GCN', no_cuda=False, noisy_level=0.3, num_hidden=128, num_proj_hidden=128, prob=0.8, prune_thr=0.2, seed=10, selection_method='cluster_degree', target_class=0, target_loss_weight=1, tau=0.1, temperature=0.5, test_model='GCN', thrd=0.5, train_lr=0.01, trigger_size=3, trojan_epochs=400, use_vs_number=True, vs_number=0, vs_ratio=0, weight_decay=0.0005)


In [5]:
if(args.dataset == 'Cora'):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.4
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.4
    args.tau = 0.1
    args.cl_lr = 0.0005
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 128
elif(args.dataset == "Pubmed"):
    args.drop_edge_rate_1 = 0.4
    args.drop_edge_rate_2 = 0.1
    args.drop_feat_rate_1 = 0.0
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256

# Certified Robustness of GNN

In [73]:
# import copy 
# # from model import UnifyModel, Encoder, Encoder_new
# from models.construct import model_construct
# from construct_graph import *
# from models.GCN_CL import GCN_Encoder

# import os.path as osp
# import random
# from time import perf_counter as t
# import yaml
# from yaml import SafeLoader

# import torch
# import torch_geometric.transforms as T
# import torch.nn.functional as F
# import torch.nn as nn
# from torch_geometric.nn import GCNConv

# from torch.distributions.bernoulli import Bernoulli
# # from torch_geometric.utils import *

# # args.cont_weight = 0
# # args.cl_num_epochs = 1200
# data = data.to(device)
# model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=0.01, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)

# for i in range(200):
#     rand_noise_data = copy.deepcopy(data)
#     rand_noise_data.edge_weight = torch.ones([rand_noise_data.edge_index.shape[1],]).to(device)
#     # m = Bernoulli(torch.tensor([args.prob]).to(device))
#     # # sample edge_index of train node
#     # train_edge_index, _, edge_mask = subgraph(data.train_mask,data.edge_index,relabel_nodes=False)
#     # train_edge_weights = torch.ones([1,train_edge_index.shape[1]]).to(device)
#     # # generate random noise 
#     # mask = m.sample(train_edge_weights.shape).squeeze(-1).int()
#     # rand_inputs = torch.randint_like(train_edge_weights, low=0, high=2, device='cuda').squeeze().int().to(device)
#     # noisy_train_edge_weights = train_edge_weights * mask + rand_inputs * (1-mask)
#     # noisy_train_edge_weights
#     for idx in idx_train:
#         idx_s = (rand_noise_data.edge_index[0] == idx).nonzero().flatten()
#         m = Bernoulli(torch.tensor([args.prob]).to(device))
#         # print(rand_noise_data.edge_weight[idx_s])
#         # print(rand_noise_data.edge_weight)
#         # break
#         mask = m.sample(rand_noise_data.edge_weight[idx_s].shape).squeeze(-1).int()
#         rand_inputs = torch.randint_like(rand_noise_data.edge_weight[idx_s], low=0, high=2, device='cuda').squeeze().int().to(device)
#         # print(rand_noise_data.edge_weight.shape,mask.shape)
#         rand_noise_data.edge_weight[idx_s] = rand_noise_data.edge_weight[idx_s] * mask + rand_inputs * (1-mask)
#         # print(rand_noise_data.edge_weight.shape)
#         # break

#     if(rand_noise_data.edge_weight!=None):
#         rand_noise_data.edge_index = rand_noise_data.edge_index[:,rand_noise_data.edge_weight.long()]
#         rand_noise_data.edge_weight = torch.ones([rand_noise_data.edge_index.shape[1],]).to(device)
    

## Smooth 

### Evasion

#### Clean graph

In [89]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation,label_classification_origin
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)

accs = []
for seed in seeds:
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Smoothed_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    rs_data = copy.deepcopy(data)
    rs_data.edge_index,rs_data.edge_weight = model.sample_noise(data.edge_index,data.edge_weight,idx_clean_test)
    z = model(rs_data.x, rs_data.edge_index,rs_data.edge_weight)

    # label_classification(z, noisy_data.y, idx_train, idx_clean_test)
    # label_classification_origin(z, rs_data.y, ratio=0.1)
    acc = label_evaluation(z, rs_data.y, idx_train, idx_clean_test)
    print(acc)
    accs.append(acc)
    # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print(acc_test)
    
    # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
    # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
    # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print("Accuracy of GNN+CL: {}".format(acc_cl))
    # final_cl_acc_noisy.append(acc_cl)
    # gnn_model = model_construct(args,'GCN',noisy_data,device)
    # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    # print(clean_acc)
    # final_gnn_acc_noisy.append(clean_acc)

accs = np.array(accs)
aver_acc = accs.mean()
print("Average Acc",aver_acc)

print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 88648])
add edge: torch.Size([2, 115170])
remove edge: torch.Size([2, 62380])
updated graph: torch.Size([2, 88902])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 10.582324028015137
Epoch 10, training loss: 10.567731857299805
Epoch 20, training loss: 10.22774600982666
Epoch 30, training loss: 9.756360054016113
Epoch 40, training loss: 9.556083679199219
Epoch 50, training loss: 9.40704345703125
Epoch 60, training loss: 9.300192832946777
Epoch 70, training loss: 9.246159553527832
Epoch 80, training loss: 9.105252265930176
Epoch 90, training loss: 9.10000228881836
Epoch 100, training loss: 8.974137306213379
Epoch 110, training loss: 8.988052368164062
Epoch 120, training loss: 8.724501609802246
Epoch 130, training loss: 8.751663208007812
Epoch 140, training loss: 8.66407299041748
Epoch 150, training loss: 8.568106651306152
Epoch 160, training loss: 8.49979305267334
Epoch 170, training loss: 8.45909309387207
Epoch 180, training loss: 8.52410

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Random Noise

In [3]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



noisy_levels = [0.1,0.2,0.3]
for nl in noisy_levels:
    args.noisy_level = nl

    accs = []
    final_cl_acc_noisy = []
    final_gnn_acc_noisy = []
    print("=== Noisy graph ===")
    rs = np.random.RandomState(args.seed)
    seeds = rs.randint(1000,size=3)
    for seed in seeds:
        # np.random.seed(seed)
        # torch.manual_seed(seed)
        # torch.cuda.manual_seed(seed)
        '''Transductive'''
        # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
        model = Smoothed_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
        model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
        
        model.eval()
        noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
        z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)

        # label_classification(z, noisy_data.y, idx_train, idx_clean_test)
        acc = label_evaluation(z, noisy_data.y, idx_train, idx_clean_test)
        print(acc)
        accs.append(acc)
        # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
        # print(acc_test)
        
        # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
        # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
        # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
        # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
        # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
        # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
        # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
        # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
        # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
        # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
        # print("Accuracy of GNN+CL: {}".format(acc_cl))
        # final_cl_acc_noisy.append(acc_cl)
        # gnn_model = model_construct(args,'GCN',noisy_data,device)
        # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
        # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
        # print(clean_acc)
        # final_gnn_acc_noisy.append(clean_acc)

    accs = np.array(accs)
    aver_acc = accs.mean()
    print("Average Acc",aver_acc)

    print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
                .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 88648])
add edge: torch.Size([2, 115128])
remove edge: torch.Size([2, 62052])
updated graph: torch.Size([2, 88532])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 10.582182884216309
Epoch 10, training loss: 9.940475463867188
Epoch 20, training loss: 9.29786491394043
Epoch 30, training loss: 9.15560531616211
Epoch 40, training loss: 8.9435453414917
Epoch 50, training loss: 8.702537536621094
Epoch 60, training loss: 8.402555465698242
Epoch 70, training loss: 8.218239784240723
Epoch 80, training loss: 7.963873863220215
Epoch 90, training loss: 7.768223762512207
Epoch 100, training loss: 7.769615173339844
Epoch 110, training loss: 7.499968528747559
Epoch 120, training loss: 7.416024208068848
Epoch 130, training loss: 7.360006809234619
Epoch 140, training loss: 7.269993782043457
Epoch 150, training loss: 7.312026023864746
Epoch 160, training loss: 7.14517879486084
Epoch 170, training loss: 7.129061698913574
Epoch 180, training loss: 7.083222

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 10, training loss: 9.828932762145996
Epoch 20, training loss: 9.283323287963867
Epoch 30, training loss: 9.094624519348145
Epoch 40, training loss: 8.922249794006348
Epoch 50, training loss: 8.64044189453125
Epoch 60, training loss: 8.403368949890137
Epoch 70, training loss: 8.161100387573242
Epoch 80, training loss: 7.992753982543945
Epoch 90, training loss: 7.811520576477051
Epoch 100, training loss: 7.733414649963379
Epoch 110, training loss: 7.662360191345215
Epoch 120, training loss: 7.617568492889404
Epoch 130, training loss: 7.536978244781494
Epoch 140, training loss: 7.444158554077148
Epoch 150, training loss: 7.336180686950684
Epoch 160, training loss: 7.275544166564941
Epoch 170, training loss: 7.228104114532471
Epoch 180, training loss: 7.11016845703125
Epoch 190, training loss: 7.150251865386963
Epoch 200, training loss: 7.103412628173828
Epoch 210, training loss: 7.056257247924805
Epoch 220, training loss: 6.924654960632324
Epoch 230, training loss: 6.905525684356689

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 10, training loss: 9.931246757507324
Epoch 20, training loss: 9.163073539733887
Epoch 30, training loss: 8.744549751281738
Epoch 40, training loss: 8.388415336608887
Epoch 50, training loss: 8.130489349365234
Epoch 60, training loss: 7.97434663772583
Epoch 70, training loss: 7.879124164581299
Epoch 80, training loss: 7.654375076293945
Epoch 90, training loss: 7.596944332122803
Epoch 100, training loss: 7.430166721343994
Epoch 110, training loss: 7.407067775726318
Epoch 120, training loss: 7.281792163848877
Epoch 130, training loss: 7.237255573272705
Epoch 140, training loss: 7.155829906463623
Epoch 150, training loss: 7.058776378631592
Epoch 160, training loss: 7.059782028198242
Epoch 170, training loss: 6.972135543823242
Epoch 180, training loss: 6.935940742492676
Epoch 190, training loss: 6.896544933319092
Epoch 200, training loss: 6.967309474945068
Epoch 210, training loss: 6.883838653564453
Epoch 220, training loss: 6.9334330558776855
Epoch 230, training loss: 6.9096832275390

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Nettack

In [3]:
from dataset import Dataset, get_PtbAdj
from torch_geometric.utils import from_scipy_sparse_matrix
if args.attack == 'random':
    from deeprobust.graph.global_attack import Random
    import random
    random.seed(15)
    attacker = Random()
    n_perturbations = int(args.ptb_rate * (adj.sum()//2))
    attacker.attack(adj, n_perturbations, type=args.r_type)
    perturbed_adj = attacker.modified_adj
    file_path = "./data/{}/{}_{}_adj_{}.npz".format(args.label_rate,args.dataset,args.attack,args.ptb_rate)
    sp.save_npz(file_path,perturbed_adj.tocsr())
if args.attack in ['meta','nettack']:
    perturbed_adj = get_PtbAdj(root="./data/{}/{}".format(0.1,'nettack'),
            name=args.dataset,
            attack_method=args.attack,
            ptb_rate=0.1)

perturbed_nodes = [938, 1689, 1802, 2185, 2003, 221, 1769, 672, 1508, 1887, 2171, 123, 1106, 688, 928, 744, 1199, 1956, 155, 2304, 656, 833, 205, 1345, 1901, 856, 147, 1091, 1904, 1083, 1977, 1015, 706, 925, 1709, 908, 1195, 2034, 484, 511, 2302, 1601, 52, 767, 1624, 443, 961, 1929, 1599, 1753, 1522, 949, 703, 2480, 1707, 1708, 710, 429, 1918, 713, 1309, 723, 1187, 1432, 230, 636, 1167, 1553, 1413, 902, 2152, 650, 479, 1913, 2233, 455, 1061, 1993, 727, 843, 117, 791, 1970, 1935, 1168, 2268, 743, 367, 1939, 2181, 2247, 1961, 365, 2291, 1384, 1981, 225, 554, 1631, 1994, 383, 2240, 2294, 776, 1700, 295, 314, 732, 2387, 1293, 2068, 826, 1273, 943, 1313, 1756, 792, 1736, 377, 2309, 821, 2324, 1585, 551, 2388, 684, 1937, 1727, 1105, 746, 7, 1852, 1566, 2316, 1458, 633, 1673, 2164, 1567, 1062, 1991, 1551, 789, 1716, 73, 1569, 1411, 1139, 2301, 877, 1435, 2087, 356, 870, 507, 837, 581, 860, 102, 444, 1253, 1840, 1604, 1592, 1878, 1131, 1161, 725, 285, 513, 2052, 712, 520, 58, 806, 919, 1028, 793, 80, 2359, 2102, 1077, 1763, 1146, 116, 777, 64, 1378, 290, 335, 400, 1454, 2389, 829, 2208, 515, 1932, 1816]
idx_perturn_test = torch.LongTensor(np.array(perturbed_nodes)).to(device)


from torch_geometric.utils import dense_to_sparse
    
# noisy_edge_index, noisy_edge_weight = dense_to_sparse(perturbed_adj)
perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(perturbed_adj)

perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)
for seed in seeds:
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Smoothed_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
    z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)
    
    label_classification(z, noisy_data.y, idx_train, idx_perturn_test)



raw graph: torch.Size([2, 10138])
add edge: torch.Size([2, 12232])
remove edge: torch.Size([2, 8032])
updated graph: torch.Size([2, 10126])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 8.703492164611816
Epoch 10, training loss: 8.51906967163086
Epoch 20, training loss: 8.504862785339355
Epoch 30, training loss: 8.502158164978027
Epoch 40, training loss: 8.494990348815918
Epoch 50, training loss: 8.48270034790039
Epoch 60, training loss: 8.408185958862305
Epoch 70, training loss: 8.285371780395508
Epoch 80, training loss: 8.105262756347656
Epoch 90, training loss: 7.871035099029541
Epoch 100, training loss: 7.654561519622803
Epoch 110, training loss: 7.557027816772461
Epoch 120, training loss: 7.469869136810303
Epoch 130, training loss: 7.334343433380127
Epoch 140, training loss: 7.195681571960449
Epoch 150, training loss: 7.066458702087402
Epoch 160, training loss: 7.038625240325928
Epoch 170, training loss: 7.102508544921875
Epoch 180, training loss: 6.961223

### Poison

In [ ]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Smoothed_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
    z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)

    label_classification(z, noisy_data.y, idx_train, idx_clean_test)

    # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print(acc_test)
    
    # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
    # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
    # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print("Accuracy of GNN+CL: {}".format(acc_cl))
    # final_cl_acc_noisy.append(acc_cl)
    # gnn_model = model_construct(args,'GCN',noisy_data,device)
    # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    # print(clean_acc)
    # final_gnn_acc_noisy.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

## Grace

### Evasion

#### Clean Graph

In [6]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder, Grace_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation,label_classification_origin
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()

accs = []

final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Grace_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_all(noisy_data.edge_index,noisy_data.edge_weight)
    z = model(data.x, data.edge_index,data.edge_weight)

    # label_classification(z, noisy_data.y, idx_train, idx_clean_test)
    acc = label_evaluation(z, data.y, idx_train, idx_clean_test)
    # label_classification_origin(z, data.y, ratio=0.1)
    print(acc)
    accs.append(acc)

    # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print(acc_test)
    
    # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
    # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
    # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print("Accuracy of GNN+CL: {}".format(acc_cl))
    # final_cl_acc_noisy.append(acc_cl)
    # gnn_model = model_construct(args,'GCN',noisy_data,device)
    # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    # print(clean_acc)
    # final_gnn_acc_noisy.append(clean_acc)
accs = np.array(accs)
aver_acc = accs.mean()
print("Average Acc",aver_acc)

print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 88648])
add edge: torch.Size([2, 115166])
remove edge: torch.Size([2, 62070])
updated graph: torch.Size([2, 88588])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 10.582009315490723
Epoch 10, training loss: 10.15438461303711
Epoch 20, training loss: 9.154176712036133
Epoch 30, training loss: 8.637389183044434
Epoch 40, training loss: 8.243880271911621
Epoch 50, training loss: 8.063199043273926
Epoch 60, training loss: 7.810876369476318
Epoch 70, training loss: 7.744261264801025
Epoch 80, training loss: 7.68227481842041
Epoch 90, training loss: 7.5518364906311035
Epoch 100, training loss: 7.406668663024902
Epoch 110, training loss: 7.358707904815674
Epoch 120, training loss: 7.275656700134277
Epoch 130, training loss: 7.218758583068848
Epoch 140, training loss: 7.175873756408691
Epoch 150, training loss: 7.145968914031982
Epoch 160, training loss: 7.136718273162842
Epoch 170, training loss: 6.945952415466309
Epoch 180, training loss: 6.9

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Random Noise

In [5]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder, Grace_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



noisy_levels = [0.1,0.2,0.3]
for nl in noisy_levels:
    args.noisy_level = nl
    accs = []

    final_cl_acc_noisy = []
    final_gnn_acc_noisy = []
    print("=== Noisy graph ===")
    rs = np.random.RandomState(args.seed)
    seeds = rs.randint(1000,size=3)
    for seed in seeds:
        np.random.seed(seed)
        # torch.manual_seed(seed)
        # torch.cuda.manual_seed(seed)
        '''Transductive'''
        # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
        model = Grace_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
        model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
        
        model.eval()
        # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_all(noisy_data.edge_index,noisy_data.edge_weight)
        z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)

        # label_classification(z, noisy_data.y, idx_train, idx_clean_test)
        acc = label_evaluation(z, noisy_data.y, idx_train, idx_clean_test)
        print(acc)
        accs.append(acc)

        # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
        # print(acc_test)
        
        # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
        # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
        # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
        # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
        # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
        # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
        # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
        # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
        # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
        # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
        # print("Accuracy of GNN+CL: {}".format(acc_cl))
        # final_cl_acc_noisy.append(acc_cl)
        # gnn_model = model_construct(args,'GCN',noisy_data,device)
        # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
        # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
        # print(clean_acc)
        # final_gnn_acc_noisy.append(clean_acc)
    accs = np.array(accs)
    aver_acc = accs.mean()
    print("Average Acc",aver_acc)

    print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
                .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 88648])
add edge: torch.Size([2, 115254])
remove edge: torch.Size([2, 62152])
updated graph: torch.Size([2, 88758])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 10.581932067871094
Epoch 10, training loss: 10.168401718139648
Epoch 20, training loss: 9.034066200256348
Epoch 30, training loss: 8.765178680419922
Epoch 40, training loss: 8.3590087890625
Epoch 50, training loss: 8.094452857971191
Epoch 60, training loss: 7.926497459411621
Epoch 70, training loss: 7.792897701263428
Epoch 80, training loss: 7.720279693603516
Epoch 90, training loss: 7.741572856903076
Epoch 100, training loss: 7.640817642211914
Epoch 110, training loss: 7.514565467834473
Epoch 120, training loss: 7.412065029144287
Epoch 130, training loss: 7.284327983856201
Epoch 140, training loss: 7.137639999389648
Epoch 150, training loss: 7.100210189819336
Epoch 160, training loss: 6.979754447937012
Epoch 170, training loss: 6.890857219696045
Epoch 180, training loss: 6.73

/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 10, training loss: 10.0530366897583
Epoch 20, training loss: 8.995680809020996
Epoch 30, training loss: 8.659631729125977
Epoch 40, training loss: 8.304037094116211
Epoch 50, training loss: 8.168578147888184
Epoch 60, training loss: 7.9080071449279785
Epoch 70, training loss: 7.778085231781006
Epoch 80, training loss: 7.695107936859131
Epoch 90, training loss: 7.664538383483887
Epoch 100, training loss: 7.586977481842041
Epoch 110, training loss: 7.449990749359131
Epoch 120, training loss: 7.307231903076172
Epoch 130, training loss: 7.184108257293701
Epoch 140, training loss: 7.055363178253174
Epoch 150, training loss: 7.038372993469238
Epoch 160, training loss: 6.920124530792236
Epoch 170, training loss: 6.84774112701416
Epoch 180, training loss: 6.7449259757995605
Epoch 190, training loss: 6.604953765869141
Epoch 200, training loss: 6.615955829620361
Epoch 210, training loss: 6.558337688446045
Epoch 220, training loss: 6.449889183044434
Epoch 230, training loss: 6.4281792640686

#### Nettack

In [ ]:
from dataset import Dataset, get_PtbAdj
from torch_geometric.utils import from_scipy_sparse_matrix
if args.attack == 'random':
    from deeprobust.graph.global_attack import Random
    import random
    random.seed(15)
    attacker = Random()
    n_perturbations = int(args.ptb_rate * (adj.sum()//2))
    attacker.attack(adj, n_perturbations, type=args.r_type)
    perturbed_adj = attacker.modified_adj
    file_path = "./data/{}/{}_{}_adj_{}.npz".format(args.label_rate,args.dataset,args.attack,args.ptb_rate)
    sp.save_npz(file_path,perturbed_adj.tocsr())
if args.attack in ['meta','nettack']:
    perturbed_adj = get_PtbAdj(root="./data/{}/{}".format(0.1,'nettack'),
            name=args.dataset,
            attack_method=args.attack,
            ptb_rate=0.1)

perturbed_nodes = [938, 1689, 1802, 2185, 2003, 221, 1769, 672, 1508, 1887, 2171, 123, 1106, 688, 928, 744, 1199, 1956, 155, 2304, 656, 833, 205, 1345, 1901, 856, 147, 1091, 1904, 1083, 1977, 1015, 706, 925, 1709, 908, 1195, 2034, 484, 511, 2302, 1601, 52, 767, 1624, 443, 961, 1929, 1599, 1753, 1522, 949, 703, 2480, 1707, 1708, 710, 429, 1918, 713, 1309, 723, 1187, 1432, 230, 636, 1167, 1553, 1413, 902, 2152, 650, 479, 1913, 2233, 455, 1061, 1993, 727, 843, 117, 791, 1970, 1935, 1168, 2268, 743, 367, 1939, 2181, 2247, 1961, 365, 2291, 1384, 1981, 225, 554, 1631, 1994, 383, 2240, 2294, 776, 1700, 295, 314, 732, 2387, 1293, 2068, 826, 1273, 943, 1313, 1756, 792, 1736, 377, 2309, 821, 2324, 1585, 551, 2388, 684, 1937, 1727, 1105, 746, 7, 1852, 1566, 2316, 1458, 633, 1673, 2164, 1567, 1062, 1991, 1551, 789, 1716, 73, 1569, 1411, 1139, 2301, 877, 1435, 2087, 356, 870, 507, 837, 581, 860, 102, 444, 1253, 1840, 1604, 1592, 1878, 1131, 1161, 725, 285, 513, 2052, 712, 520, 58, 806, 919, 1028, 793, 80, 2359, 2102, 1077, 1763, 1146, 116, 777, 64, 1378, 290, 335, 400, 1454, 2389, 829, 2208, 515, 1932, 1816]
idx_perturn_test = torch.LongTensor(np.array(perturbed_nodes)).to(device)


from torch_geometric.utils import dense_to_sparse
    
# noisy_edge_index, noisy_edge_weight = dense_to_sparse(perturbed_adj)
perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(perturbed_adj)
perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)

import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Grace_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Grace_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_all(noisy_data.edge_index,noisy_data.edge_weight)
    z = model(noisy_data.x, perturb_edge_index,perturb_edge_weight)

    label_classification(z, noisy_data.y, idx_train, idx_perturn_test)

# for seed in seeds:
#     # np.random.seed(seed)
#     # torch.manual_seed(seed)
#     # torch.cuda.manual_seed(seed)
#     '''Transductive'''
#     # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
#     model = Smoothed_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
#     model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
#     model.eval()
#     noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
#     z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)

#     label_classification(z, noisy_data.y, idx_train, idx_perturn_test)


### Poison

In [ ]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Smoothed_Encoder, Grace_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification
# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = Grace_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    model.eval()
    # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_all(noisy_data.edge_index,noisy_data.edge_weight)
    z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)

    label_classification(z, noisy_data.y, idx_train, idx_clean_test)

    # acc_test = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print(acc_test)
    
    # # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    # #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
    # # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
    # # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    # # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    # acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    # print("Accuracy of GNN+CL: {}".format(acc_cl))
    # final_cl_acc_noisy.append(acc_cl)
    # gnn_model = model_construct(args,'GCN',noisy_data,device)
    # gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    # clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    # print(clean_acc)
    # final_gnn_acc_noisy.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

## GCN

In [ ]:
from dataset import Dataset, get_PtbAdj
from torch_geometric.utils import from_scipy_sparse_matrix
if args.attack == 'random':
    from deeprobust.graph.global_attack import Random
    import random
    random.seed(15)
    attacker = Random()
    n_perturbations = int(args.ptb_rate * (adj.sum()//2))
    attacker.attack(adj, n_perturbations, type=args.r_type)
    perturbed_adj = attacker.modified_adj
    file_path = "./data/{}/{}_{}_adj_{}.npz".format(args.label_rate,args.dataset,args.attack,args.ptb_rate)
    sp.save_npz(file_path,perturbed_adj.tocsr())
if args.attack in ['meta','nettack']:
    perturbed_adj = get_PtbAdj(root="./data/{}/{}".format(0.1,'nettack'),
            name=args.dataset,
            attack_method=args.attack,
            ptb_rate=0.1)

perturbed_nodes = [938, 1689, 1802, 2185, 2003, 221, 1769, 672, 1508, 1887, 2171, 123, 1106, 688, 928, 744, 1199, 1956, 155, 2304, 656, 833, 205, 1345, 1901, 856, 147, 1091, 1904, 1083, 1977, 1015, 706, 925, 1709, 908, 1195, 2034, 484, 511, 2302, 1601, 52, 767, 1624, 443, 961, 1929, 1599, 1753, 1522, 949, 703, 2480, 1707, 1708, 710, 429, 1918, 713, 1309, 723, 1187, 1432, 230, 636, 1167, 1553, 1413, 902, 2152, 650, 479, 1913, 2233, 455, 1061, 1993, 727, 843, 117, 791, 1970, 1935, 1168, 2268, 743, 367, 1939, 2181, 2247, 1961, 365, 2291, 1384, 1981, 225, 554, 1631, 1994, 383, 2240, 2294, 776, 1700, 295, 314, 732, 2387, 1293, 2068, 826, 1273, 943, 1313, 1756, 792, 1736, 377, 2309, 821, 2324, 1585, 551, 2388, 684, 1937, 1727, 1105, 746, 7, 1852, 1566, 2316, 1458, 633, 1673, 2164, 1567, 1062, 1991, 1551, 789, 1716, 73, 1569, 1411, 1139, 2301, 877, 1435, 2087, 356, 870, 507, 837, 581, 860, 102, 444, 1253, 1840, 1604, 1592, 1878, 1131, 1161, 725, 285, 513, 2052, 712, 520, 58, 806, 919, 1028, 793, 80, 2359, 2102, 1077, 1763, 1146, 116, 777, 64, 1378, 290, 335, 400, 1454, 2389, 829, 2208, 515, 1932, 1816]
idx_perturn_test = torch.LongTensor(np.array(perturbed_nodes)).to(device)

from torch_geometric.utils import dense_to_sparse
    
# noisy_edge_index, noisy_edge_weight = dense_to_sparse(perturbed_adj)
perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(perturbed_adj)
perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)

from models.GCN import GCN
from models.construct import model_construct
gnn_model = model_construct(args,'GCN',data,device)

gnn_model.fit(data.x, data.edge_index, None, data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
perturb_acc = gnn_model.test(data.x,perturb_edge_index,perturb_edge_weight,data.y,idx_perturn_test)
print(perturb_acc)

## Target attack

In [4]:
from eval import label_classification, label_evaluation

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--dataset', type=str, default='cora', choices=['cora', 'cora_ml', 'citeseer','Pubmed'], help='dataset')
parser.add_argument('--ptb_rate', type=float, default=0.1,  help='pertubation rate')
parser.add_argument("--label_rate", type=float, default=0.1, help='rate of labeled data')
args = parser.parse_known_args()[0]
args.cuda = torch.cuda.is_available()
print('cuda: %s' % args.cuda)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

np.random.seed(15)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# data = Dataset(root='./tmp/', name="cora")
data = Dataset(root='/tmp/', name='cora') # load clean graph
adj, features, labels = data.adj, data.features, data.labels

# idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# idx_train = idx_train[:int(args.label_rate * adj.shape[0])]
# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device=device)

surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)

def test(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train, idx_val, patience=10)

    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    acc_test = accuracy(output[idx_test], labels[idx_test])

    print("Overall test set results:",
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

def multi_test_poison():
    # test on 40 nodes on poisoining attack
    cnt = 0
    degrees = adj.sum(0).A1
    np.random.seed(42)
    idx = np.arange(0,adj.shape[0])
    np.random.shuffle(idx)
    node_list = idx[:int(args.ptb_rate*len(idx))]

    num = len(node_list)
    print('=== [Poisoning] Attacking %s nodes respectively ===' % num)

    modified_adj = adj
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
        model = model.to(device)

        model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
        modified_adj = model.modified_adj
        modified_features = model.modified_features
        acc = single_test(modified_adj, modified_features, target_node)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))
    import os
    import scipy.sparse as sp
    path = os.path.join("./data/{}".format(args.label_rate),"nettack/")
    if not os.path.exists(path):
        os.makedirs(path)
    file_path = os.path.join(path,"{}.npz".format(args.dataset))
    if type(modified_adj) is torch.Tensor:
        sparse_adj = to_scipy(modified_adj)
        sp.save_npz(file_path, sparse_adj)
    else:
        sp.save_npz(file_path, modified_adj)



def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

#%%
cnt = 0
degrees = adj.sum(0).A1
np.random.seed(42)
# idx = np.arange(0,adj.shape[0])
idx = np.array(idx_clean_test.cpu())
print("idx:",idx)
np.random.shuffle(idx)
node_list = idx[:int(args.ptb_rate*len(idx))]
# node_list = idx[:50]
# node_list=[0]

modified_adj = adj
num = len(node_list)

cl_cnt=0

print('=== [Poisoning] Attacking %s nodes respectively ===' % num)
for target_node in tqdm(node_list):
    n_perturbations = int(degrees[target_node])
    atk_model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    atk_model = atk_model.to(device)
    atk_model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
    modified_adj = atk_model.modified_adj
    modified_features = atk_model.modified_features
    acc = single_test(modified_adj, modified_features, target_node)
    print("GCN",acc)
    if acc == 0:
        cnt += 1

    perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(modified_adj)
    perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)
    
    perturb_edge_index,perturb_edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
    z = model(noisy_data.x, perturb_edge_index,perturb_edge_weight)
    idx_target = torch.LongTensor([target_node]).to(device)
    cl_acc = label_evaluation(z, noisy_data.y, idx_train, idx_target)
    print("CL",cl_acc)
    if cl_acc == 0:
        cl_cnt += 1
    # break
    
print('misclassification rate : %s' % (cnt/num))
print('misclassification rate : %s' % (cl_cnt/num))

cuda: True
Loading cora dataset...
Selecting 1 largest connected components
idx: [   0    1    2 ... 2482 2483 2484]
=== [Poisoning] Attacking 198 nodes respectively ===


  0%|          | 0/198 [00:00<?, ?it/s]/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../anaconda3/envs/py38_torch120/lib/python3.8/site-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
  1%|          | 1/198 [00:03<11:44,  3.58s/it]

CL 1.0
GCN False


...

CL 1.0
GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 99%|█████████▉| 197/198 [04:50<00:01,  1.57s/it]

CL 1.0
GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 198/198 [04:52<00:00,  1.48s/it]

CL 1.0
misclassification rate : 0.8737373737373737
misclassification rate : 0.21717171717171718


In [9]:
from eval import label_classification, label_evaluation

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--dataset', type=str, default='cora', choices=['cora', 'cora_ml', 'citeseer','Pubmed'], help='dataset')
parser.add_argument('--ptb_rate', type=float, default=0.1,  help='pertubation rate')
parser.add_argument("--label_rate", type=float, default=0.1, help='rate of labeled data')
args = parser.parse_known_args()[0]
args.cuda = torch.cuda.is_available()
print('cuda: %s' % args.cuda)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

np.random.seed(15)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# data = Dataset(root='./tmp/', name="cora")
data = Dataset(root='/tmp/', name='cora') # load clean graph
adj, features, labels = data.adj, data.features, data.labels

# idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# idx_train = idx_train[:int(args.label_rate * adj.shape[0])]
# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device=device)

surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)

def test(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train, idx_val, patience=10)

    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    acc_test = accuracy(output[idx_test], labels[idx_test])

    print("Overall test set results:",
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

def multi_test_poison():
    # test on 40 nodes on poisoining attack
    cnt = 0
    degrees = adj.sum(0).A1
    np.random.seed(42)
    idx = np.arange(0,adj.shape[0])
    np.random.shuffle(idx)
    node_list = idx[:int(args.ptb_rate*len(idx))]

    num = len(node_list)
    print('=== [Poisoning] Attacking %s nodes respectively ===' % num)

    modified_adj = adj
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
        model = model.to(device)

        model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
        modified_adj = model.modified_adj
        modified_features = model.modified_features
        acc = single_test(modified_adj, modified_features, target_node)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))
    import os
    import scipy.sparse as sp
    path = os.path.join("./data/{}".format(args.label_rate),"nettack/")
    if not os.path.exists(path):
        os.makedirs(path)
    file_path = os.path.join(path,"{}.npz".format(args.dataset))
    if type(modified_adj) is torch.Tensor:
        sparse_adj = to_scipy(modified_adj)
        sp.save_npz(file_path, sparse_adj)
    else:
        sp.save_npz(file_path, modified_adj)



def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

#%%
cnt = 0
degrees = adj.sum(0).A1
np.random.seed(42)
# idx = np.arange(0,adj.shape[0])
idx = np.array(idx_clean_test.cpu())
print("idx:",idx)
np.random.shuffle(idx)
node_list = idx[:int(args.ptb_rate*len(idx))]
# node_list = idx[:50]
# node_list=[0]

modified_adj = adj
num = len(node_list)

cl_cnt=0

print('=== [Poisoning] Attacking %s nodes respectively ===' % num)
for target_node in tqdm(node_list):
    n_perturbations = int(degrees[target_node])
    atk_model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    atk_model = atk_model.to(device)
    atk_model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
    modified_adj = atk_model.modified_adj
    modified_features = atk_model.modified_features
    acc = single_test(modified_adj, modified_features, target_node)
    print("GCN",acc)
    if acc == 0:
        cnt += 1

    perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(modified_adj)
    perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)
    
    perturb_edge_index,perturb_edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
    z = model(noisy_data.x, perturb_edge_index,perturb_edge_weight)
    idx_target = torch.LongTensor([target_node]).to(device)
    cl_acc = label_evaluation(z, noisy_data.y, idx_train, idx_target)
    print("CL",cl_acc)
    if cl_acc == 0:
        cl_cnt += 1
    # break
    
print('misclassification rate : %s' % (cnt/num))
print('misclassification rate : %s' % (cl_cnt/num))

cuda: True
Loading cora dataset...
Selecting 1 largest connected components
idx: [   0    1    2 ... 2482 2483 2484]
=== [Poisoning] Attacking 198 nodes respectively ===


  0%|          | 0/198 [00:00<?, ?it/s]/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../anaconda3/envs/py38_torch120/lib/python3.8/site-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
  1%|          | 1/198 [00:03<11:32,  3.52s/it]

CL 1.0
GCN False


...

CL 1.0
GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 99%|█████████▉| 197/198 [04:40<00:01,  1.48s/it]

CL 1.0
GCN False


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 198/198 [04:42<00:00,  1.42s/it]

CL 1.0
misclassification rate : 0.8737373737373737
misclassification rate : 0.21717171717171718


Grace Encoder with NEttack

In [12]:
from eval import label_classification, label_evaluation

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

from torch_geometric.utils import from_scipy_sparse_matrix
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--dataset', type=str, default='cora', choices=['cora', 'cora_ml', 'citeseer','Pubmed'], help='dataset')
parser.add_argument('--ptb_rate', type=float, default=0.1,  help='pertubation rate')
parser.add_argument("--label_rate", type=float, default=0.1, help='rate of labeled data')
args = parser.parse_known_args()[0]
args.cuda = torch.cuda.is_available()
print('cuda: %s' % args.cuda)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

np.random.seed(15)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# data = Dataset(root='./tmp/', name="cora")
data = Dataset(root='/tmp/', name='pubmed') # load clean graph
adj, features, labels = data.adj, data.features, data.labels

num_nodes = data.adj.shape[0]
num_classes = len(np.unique(data.labels))
idx_test = torch.tensor(np.array(range(num_nodes-1000,num_nodes))).to(device)
idx_train = torch.tensor(np.array(range(0,num_classes*20))).to(device)
idx_val = torch.tensor(np.array(range(num_classes*20,num_classes*20+500))).to(device)

# idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# idx_train = idx_train[:int(args.label_rate * adj.shape[0])]
# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device=device)

surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)

def test(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train, idx_val, patience=10)

    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    acc_test = accuracy(output[idx_test], labels[idx_test])

    print("Overall test set results:",
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

def multi_test_poison():
    # test on 40 nodes on poisoining attack
    cnt = 0
    degrees = adj.sum(0).A1
    np.random.seed(42)
    idx = np.arange(0,adj.shape[0])
    np.random.shuffle(idx)
    node_list = idx[:int(args.ptb_rate*len(idx))]

    num = len(node_list)
    print('=== [Poisoning] Attacking %s nodes respectively ===' % num)

    modified_adj = adj
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
        model = model.to(device)

        model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
        modified_adj = model.modified_adj
        modified_features = model.modified_features
        acc = single_test(modified_adj, modified_features, target_node)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))
    import os
    import scipy.sparse as sp
    path = os.path.join("./data/{}".format(args.label_rate),"nettack/")
    if not os.path.exists(path):
        os.makedirs(path)
    file_path = os.path.join(path,"{}.npz".format(args.dataset))
    if type(modified_adj) is torch.Tensor:
        sparse_adj = to_scipy(modified_adj)
        sp.save_npz(file_path, sparse_adj)
    else:
        sp.save_npz(file_path, modified_adj)



def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

#%%
cnt = 0
degrees = adj.sum(0).A1
np.random.seed(42)
# idx = np.arange(0,adj.shape[0])
idx = np.array(idx_clean_test.cpu())
print("idx:",idx)
np.random.shuffle(idx)
node_list = idx[:int(args.ptb_rate*len(idx))]
# node_list = idx[:50]
# node_list=[0]

modified_adj = adj
num = len(node_list)

cl_cnt=0

print('=== [Poisoning] Attacking %s nodes respectively ===' % num)
for target_node in tqdm(node_list):
    n_perturbations = int(degrees[target_node])
    atk_model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    atk_model = atk_model.to(device)
    atk_model.attack(features, modified_adj, labels, target_node, n_perturbations, verbose=False)
    modified_adj = atk_model.modified_adj
    modified_features = atk_model.modified_features
    acc = single_test(modified_adj, modified_features, target_node)
    print("GCN",acc)
    if acc == 0:
        cnt += 1

    perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(modified_adj)
    perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device).float()
    
    # perturb_edge_index,perturb_edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
    z = model(noisy_data.x, perturb_edge_index,perturb_edge_weight)
    idx_target = torch.LongTensor([target_node]).to(device)
    cl_acc = label_evaluation(z, noisy_data.y, idx_train, idx_target)
    print("CL",cl_acc)
    if cl_acc == 0:
        cl_cnt += 1
    # break
    
print('misclassification rate : %s' % (cnt/num))
print('misclassification rate : %s' % (cl_cnt/num))

cuda: True
Loading pubmed dataset...


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/deeprobust/graph/data/dataset.py:209: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


idx: [18717 18718 18719 18720 18721 18722 18723 18724 18725 18726 18727 18728
 18729 18730 18731 18732 18733 18734 18735 18736 18737 18738 18739 18740
 18741 18742 18743 18744 18745 18746 18747 18748 18749 18750 18751 18752
 18753 18754 18755 18756 18757 18758 18759 18760 18761 18762 18763 18764
 18765 18766 18767 18768 18769 18770 18771 18772 18773 18774 18775 18776
 18777 18778 18779 18780 18781 18782 18783 18784 18785 18786 18787 18788
 18789 18790 18791 18792 18793 18794 18795 18796 18797 18798 18799 18800
 18801 18802 18803 18804 18805 18806 18807 18808 18809 18810 18811 18812
 18813 18814 18815 18816 18817 18818 18819 18820 18821 18822 18823 18824
 18825 18826 18827 18828 18829 18830 18831 18832 18833 18834 18835 18836
 18837 18838 18839 18840 18841 18842 18843 18844 18845 18846 18847 18848
 18849 18850 18851 18852 18853 18854 18855 18856 18857 18858 18859 18860
 18861 18862 18863 18864 18865 18866 18867 18868 18869 18870 18871 18872
 18873 18874 18875 18876 18877 18878 18879 188

  0%|          | 0/100 [00:00<?, ?it/s]

GCN False


...

GCN False
CL 0.0


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 99%|█████████▉| 99/100 [52:44<00:13, 13.56s/it]

GCN False
CL 0.0


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 100/100 [52:57<00:00, 31.77s/it]

GCN False
CL 1.0
misclassification rate : 0.91
misclassification rate : 0.85


In [10]:
perturb_edge_index.float()

tensor([[    0.,     0.,     0.,  ..., 19714., 19715., 19716.],
        [ 1378.,  1544.,  6092.,  ..., 12278.,  4284., 16030.]],
       device='cuda:2')

# Unify Contrastive GNN

## Structure noise

### Noisy

In [ ]:
import copy 
from model import UnifyModel, Encoder, Encoder_new
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

# args.cont_weight = 0
# args.cl_num_epochs = 1200
data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=args.cl_lr, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=0.0005, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # # encoder = Encoder_new(dataset.num_features, args.num_hidden, args.cl_activation,
    # #                         base_model=args.cl_base_model, k=args.cl_num_layers,mlp_hidden = 128, gcn_hidden=128).to(device)
    # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    print("Accuracy of GNN+CL: {}".format(acc_cl))
    final_cl_acc_noisy.append(acc_cl)
    gnn_model = model_construct(args,'GCN',noisy_data,device)
    gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    print(clean_acc)
    final_gnn_acc_noisy.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

### Raw

In [ ]:
import copy 
from model import UnifyModel
from models.construct import model_construct


import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
from model import Encoder, Model, drop_feature

data = data.to(device)
config = yaml.load(open(args.config), Loader=SafeLoader)[args.dataset]
num_epochs = config['num_epochs']
# args.cl_lr = 0.0001
# args.weight_decay = 0.00001
# args.cl_num_epochs = 3000
# args.cl_num_epochs = 500
# args.cl_lr = 0.0005
# weight_decay = config['weight_decay']
# args.seed = config['seed']
# args.cont_batch_size = config['cont_batch_size']
# args.cont_weight = config['cont_weight']
# args.add_edge_rate_1 = config['add_edge_rate_1']
# args.add_edge_rate_2 = config['add_edge_rate_2']
# args.drop_edge_rate_1 = config['drop_edge_rate_1']
# args.drop_edge_rate_2 = config['drop_edge_rate_2']
# args.drop_feat_rate_1 = config['drop_feature_rate_1']
# args.drop_feat_rate_2 = config['drop_feature_rate_2']
# args.add_edge_rate_1 = 0
# args.add_edge_rate_2 = 0
# args.drop_edge_rate_1 = 0.1
# args.drop_edge_rate_2 = 0.1
# args.drop_feat_rate_1 = 0.3
# args.drop_feat_rate_2 = 0.4
# args.cont_weight = 10
# args.weight_decay = 0.00005
num_class = int(data.y.max()+1)

# noisy_data = construct_noisy_graph(data,perturb_ratio=0.25,mode='random_noise')
# noisy_data = noisy_data.to(device)
seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()


final_cl_acc = []
final_gnn_acc = []
print("=== Raw graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(20,size=3)
# seeds = [39788]
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
    #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    # model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # model.fit(args, data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, dropout=0.5, lr=0.01, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
    model.fit(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    
    # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    acc_cl = model.test(data.x, data.edge_index,data.edge_weight,data.y,idx_clean_test)
    print("Accuracy of GNN+CL: {}".format(acc_cl))
    final_cl_acc.append(acc_cl)
    gnn_model = model_construct(args,'GCN',data,device)
    gnn_model.fit(data.x, data.edge_index, None, data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    clean_acc = gnn_model.test(data.x,data.edge_index,data.edge_weight,data.y,idx_overall_test)
    print(clean_acc)
    final_gnn_acc.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc),np.std(final_cl_acc),np.average(final_gnn_acc),np.std(final_gnn_acc)))